Este projeto é uma análise exploratória do dataset Intel Image Classification, disponível no Kaggle. O dataset contém 25 mil imagens de 6 classes diferentes, sendo elas: buildings, forest, glacier, mountain, sea e street. O objetivo é classificar as imagens de acordo com a classe a qual pertencem.



In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

2023-12-05 12:17:23.307695: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 12:17:23.350036: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 12:17:23.350067: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 12:17:23.351513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 12:17:23.357396: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 12:17:23.357844: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Preparing Dataset

In [22]:
train_dataset = keras.utils.image_dataset_from_directory(
    './seg_train', batch_size=64, image_size=(150, 150),  label_mode='categorical')

validation_dataset = keras.utils.image_dataset_from_directory(
    './seg_test', batch_size=64, image_size=(150, 150),  label_mode='categorical')

# prediction_dataset = keras.utils.image_dataset_from_directory(
#     './seg_pred', batch_size=64, image_size=(150, 150))

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


## Exploratore Analysis

Como veremos abaixo, o dataset é composto por imagens de 150x150 pixels, coloridas e com 3 canais (RGB). Além disso, o dataset é balanceado, ou seja, possui a mesma quantidade de imagens para cada classe.
Exibiremos algumas imagens do dataset para melhor entendimento.



In [3]:
# Describe some characteristics of the dataset.
print(train_dataset.class_names)
print(validation_dataset.class_names)
print(train_dataset)
print(validation_dataset)

# Visualize one of the images in the dataset.
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_dataset.class_names[labels[i]])
        plt.axis("off")

### Model Definition

In [23]:
neural_networks = [
    {
        'description': 'Conv2D_MaxPooling_Conv2D',
        'model': Sequential([
                    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
                    MaxPooling2D(pool_size=(2, 2)),
                    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'),
                    Flatten(input_shape=(150, 150, 3)),
                    Dense(units=6, activation='softmax')
                    ]
                ),
        'epochs': 10,
        # 'steps_per_epoch': 100,
        # 'validation_steps': 10,
        'optimizer': Adam(learning_rate=0.0001),
        'loss_function': 'categorical_crossentropy',
        'accuracy': None,
        'loss': None,
        'training_time': None,
    }
]

In [24]:
for nn in neural_networks:
    model = nn['model']
    print(nn)
    model.compile(optimizer=nn['optimizer'], loss=nn['loss_function'], metrics=['accuracy'])

{'description': 'Conv2D_MaxPooling_Conv2D', 'model': <keras.src.engine.sequential.Sequential object at 0x7f71d9f6f550>, 'epochs': 10, 'optimizer': <keras.src.optimizers.adam.Adam object at 0x7f71d9dd0d60>, 'loss_function': 'categorical_crossentropy', 'accuracy': None, 'loss': None, 'training_time': None}


In [25]:
for nn in neural_networks:
    model = nn['model']
    model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=nn['epochs'],
        verbose=2
    )


Epoch 1/10


220/220 - 212s - loss: 27.3915 - accuracy: 0.4697 - val_loss: 3.3073 - val_accuracy: 0.5683 - 212s/epoch - 966ms/step
Epoch 2/10
220/220 - 198s - loss: 1.7071 - accuracy: 0.6738 - val_loss: 2.6072 - val_accuracy: 0.5790 - 198s/epoch - 901ms/step
Epoch 3/10
220/220 - 213s - loss: 0.5504 - accuracy: 0.8422 - val_loss: 2.3823 - val_accuracy: 0.6070 - 213s/epoch - 969ms/step
Epoch 4/10
220/220 - 200s - loss: 0.2133 - accuracy: 0.9370 - val_loss: 2.5030 - val_accuracy: 0.6120 - 200s/epoch - 910ms/step
Epoch 5/10
220/220 - 193s - loss: 0.1035 - accuracy: 0.9726 - val_loss: 2.6042 - val_accuracy: 0.6203 - 193s/epoch - 878ms/step
Epoch 6/10
220/220 - 201s - loss: 0.0636 - accuracy: 0.9865 - val_loss: 2.6083 - val_accuracy: 0.6347 - 201s/epoch - 916ms/step
Epoch 7/10
220/220 - 209s - loss: 0.0474 - accuracy: 0.9922 - val_loss: 2.7614 - val_accuracy: 0.6117 - 209s/epoch - 951ms/step
Epoch 8/10
220/220 - 237s - loss: 0.0373 - accuracy: 0.9947 - val_loss: 2.7442 - val_accuracy: 0.6287 - 237s/epoch

## Prediction

In [ ]:
# predictions = model.predict(prediction_dataset, verbose=2)

## Visualizing

In [ ]:
# from sklearn.metrics import confusion_matrix

# y_pred = np.argmax(predictions, axis=1)
# y_true = np.concatenate([y for x, y in prediction_dataset], axis=0)
# cm = confusion_matrix(y_true, y_pred)